In [ ]:
# Mount google drive here ..
import os
from google.colab import drive
if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive')


input_path = "/content/drive/MyDrive/ML/BERT/train.csv"
output_path = "/content/drive/MyDrive/ML/train_preprocess.csv"

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 2)

In [ ]:
data = pd.read_csv(input_path)
data.head()

,id,text,class
0,dee6dfc5,"Đến quán 2 lần thôi , rất là thích !\nQuán tuy nằm trong hẻm nhưng rất dễ tìm , quán rộng_rãi thoáng mát sạch_sẽ .\nNhân_viên phục_vụ nhiệt_tình , siêu dễ_thương .\nThực_đơn phong_phú đa_dạng , giá siêu rẻ .\nThức_ăn nêm nếm hợp_khẩu vị , ngon_miệng .\nDesert ngọt vừa , không quá ngấy , rất ngon .\nTrà sữa , uyên_ương thơm và đúng vị mình đã từng uống ở HongKong .\nNói_chung điểm 10 cho chất_lượng , thích quán lắm !",1
1,800813f5,Đến quán vào tối chủ_nhật . Có band hát . Khá ổn . Tuy_nhiên không_gian tầng 1 hơi hẹp . Ngồi cả ra vỉa_hè . Đồ uống ko ngon lămz,0
2,6553e47f,Phục_vụ lâu quá mặc_dù khách rất vắng .\nĐợi gần 15p mới mang được ly cà_phê ra . Ông này chỉ khéo quảng_bá chứ với mình chắc không quay lại_nữa .,0
3,b45a1ff1,"Ko gian bé_tí , quán chật_chội , đông người nên phục_vụ chưa tốt . Đồ uống bình_thường ( đặc_biệt khi so với giá_cả ) và ko có gì đặc_sắc cả .",0
4,f92694b0,"Khi mình order , đặt bánh thì nhận được sự tiếp_đãi , tư_vấn và nói_chuyện rất gần_gũi , rất nhiệt_tình .\nBánh thì rất tuyệt , nhân béo , rau_câu không dùng phẩm . Sau_này có dịp sinh_nhật , mình sẽ order mẫu lớn .\nChúc các bạn làm_ăn phát_đạt !",1


In [ ]:
# (optional preprocess) FIX BROKEN UNICODE WORD
# 
# Example: "Mi ̀ nh đa ̃ đê ́ n đây vơ ́ i y ́ đi ̣ nh đê ́ n"
# Result:  "Mình đã đến đây với ý định đến"
# Test with sample 14, 10667 to see the result


# Subprocess: Split 2 stuck words
# Example: "đãđến"  ->  "đã đến"
# Based on Generic Human's source code in this stackoverflow topic
# Original link: https://stackoverflow.com/a/11642687/13619642
# ------------------------------------------------------------------
# ------------------------------------------------------------------
# Download Vietnamese word list by frequency
from math import log
!wget "https://drive.google.com/uc?export=download&id=18kzZsH_hYdIZ9pQ1l7fyZcc3iHZwhuoa" -O "words.txt" -q
words_path = "/content/words.txt"
# Build a cost dictionary, assuming Zipf's law and cost = -math.log(probability).
words = open(words_path).read().split()
wordcost = dict((k, log((i+1)*log(len(words)))) for i,k in enumerate(words))
maxword = max(len(x) for x in words)

def infer_spaces(s):
    """Uses dynamic programming to infer the location of spaces in a string
    without spaces."""
    # Find the best match for the i first characters, assuming cost has
    # been built for the i-1 first characters.
    # Returns a pair (match_cost, match_length).
    def best_match(i):
        candidates = enumerate(reversed(cost[max(0, i-maxword):i]))
        return min((c + wordcost.get(s[i-k-1:i], 9e999), k+1) for k,c in candidates)
    # Build the cost array.
    cost = [0]
    for i in range(1,len(s)+1):
        c,k = best_match(i)
        cost.append(c)
    # Backtrack to recover the minimal-cost string.
    out = []
    i = len(s)
    while i>0:
        c,k = best_match(i)
        assert c == cost[i]
        out.append(s[i-k:i])
        i -= k
    # Return
    return " ".join(reversed(out))
# ------------------------------------------------------------------
# ------------------------------------------------------------------


# Main process: Fix invalid unicode word
# Example: "Mi ̀ nh đa ̃ đê ́ n"  ->  "Mình đãđến"
import unicodedata, re
def handle_invalid_unicode(text):
    # Detect invalid unicode
    if re.search(b"( )(\xcc[\x80-\xa9])( )", text.encode('utf-8')):
        # Fix invalid character
        text = re.sub(b"( )(\xcc[\x80-\xa9])( )", br'\2', text.encode('utf-8')).decode('utf-8')
        # Fix non-space word
        word_list = text.split(" ")
        stuck_indexes = []
        for i, w in enumerate(word_list):
            # Detect stuck word
            if len(re.findall(b"\xcc", w.encode('utf-8'))) > 0:
                stuck_indexes.append(i)
        text = unicodedata.normalize('NFC', text)  # Normalize text
        if len(stuck_indexes) > 0:
            normalized_word_list = text.split(" ")
            # Split stuck words. Use subprocess
            for i in stuck_indexes:
                normalized_word_list[i] = infer_spaces(normalized_word_list[i])
            text = " ".join(normalized_word_list)
    return text


In [ ]:
# Preprocess text
from bs4 import BeautifulSoup
import string, re


def preprocess_text(text):
    # Remove HTML
    soup = BeautifulSoup(text, "html5lib")
    text = soup.get_text(separator=" ")

    # Remove icon
    icon_re = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    text = icon_re.sub(r'', text)

    # Remove punctuation
    punctuation = string.punctuation.replace("_", "")  # Keep _ symbol
    text = "".join([i if i not in punctuation else " " for i in text])

    # Lowercase text
    text = text.lower()

    # Remove duplicate characters
    text = re.sub(r'(.)\1{1,}', r'\1', text)

    # Remove new_line and strip
    text = text.replace("\n", " ").strip()

    # Remove double space
    text = re.sub(" +", " ", text)

    # Fix broken unicode format 
    text = handle_invalid_unicode(text)

    # Remove _ sysbol
    text = text.replace("_", " ")

    # Return
    return text


In [ ]:
data['preprocess_text'] = data['text'].apply(lambda x: preprocess_text(x))
data.head(10)

,id,text,class,preprocess_text
0,dee6dfc5,"Đến quán 2 lần thôi , rất là thích !\nQuán tuy nằm trong hẻm nhưng rất dễ tìm , quán rộng_rãi thoáng mát sạch_sẽ .\nNhân_viên phục_vụ nhiệt_tình , siêu dễ_thương .\nThực_đơn phong_phú đa_dạng , giá siêu rẻ .\nThức_ăn nêm nếm hợp_khẩu vị , ngon_miệng .\nDesert ngọt vừa , không quá ngấy , rất ngon .\nTrà sữa , uyên_ương thơm và đúng vị mình đã từng uống ở HongKong .\nNói_chung điểm 10 cho chất_lượng , thích quán lắm !",1,đến quán 2 lần thôi rất là thích quán tuy nằm trong hẻm nhưng rất dễ tìm quán rộng rãi thoáng mát sạch sẽ nhân viên phục vụ nhiệt tình siêu dễ thương thực đơn phong phú đa dạng giá siêu rẻ thức ăn nêm nếm hợp khẩu vị ngon miệng desert ngọt vừa không quá ngấy rất ngon trà sữa uyên ương thơm và đúng vị mình đã từng uống ở hongkong nói chung điểm 10 cho chất lượng thích quán lắm
1,800813f5,Đến quán vào tối chủ_nhật . Có band hát . Khá ổn . Tuy_nhiên không_gian tầng 1 hơi hẹp . Ngồi cả ra vỉa_hè . Đồ uống ko ngon lămz,0,đến quán vào tối chủ nhật có band hát khá ổn tuy nhiên không gian tầng 1 hơi hẹp ngồi cả ra vỉa hè đồ uống ko ngon lămz
2,6553e47f,Phục_vụ lâu quá mặc_dù khách rất vắng .\nĐợi gần 15p mới mang được ly cà_phê ra . Ông này chỉ khéo quảng_bá chứ với mình chắc không quay lại_nữa .,0,phục vụ lâu quá mặc dù khách rất vắng đợi gần 15p mới mang được ly cà phê ra ông này chỉ khéo quảng bá chứ với mình chắc không quay lại nữa
3,b45a1ff1,"Ko gian bé_tí , quán chật_chội , đông người nên phục_vụ chưa tốt . Đồ uống bình_thường ( đặc_biệt khi so với giá_cả ) và ko có gì đặc_sắc cả .",0,ko gian bé tí quán chật chội đông người nên phục vụ chưa tốt đồ uống bình thường đặc biệt khi so với giá cả và ko có gì đặc sắc cả
4,f92694b0,"Khi mình order , đặt bánh thì nhận được sự tiếp_đãi , tư_vấn và nói_chuyện rất gần_gũi , rất nhiệt_tình .\nBánh thì rất tuyệt , nhân béo , rau_câu không dùng phẩm . Sau_này có dịp sinh_nhật , mình sẽ order mẫu lớn .\nChúc các bạn làm_ăn phát_đạt !",1,khi mình order đặt bánh thì nhận được sự tiếp đãi tư vấn và nói chuyện rất gần gũi rất nhiệt tình bánh thì rất tuyệt nhân béo rau câu không dùng phẩm sau này có dịp sinh nhật mình sẽ order mẫu lớn chúc các bạn làm ăn phát đạt
5,8fea8db2,"An ở đây lần thứ n mà giờ mới review . Quán lúc_nào cũng đông . Nhân_viên chạy ra chạy vô tội_nghiệp luôn . Mọi thứ viên chiên ở đây có đủ . Mỗi lần thèm an vặt là mình lại ra đây . Giá khá rẻ . 1 dĩa an phủ_phê luôn 4 đứa chia ra mỗi đứa 50k . Lúc trước an thì quán chỉ bán ở nhà chính thôi , riết rồi lấn ra đuờng rồi qua bên quán cơm tấm 3 lùn bán luôn . Phát_triển ghê_gớm",0,an ở đây lần thứ n mà giờ mới review quán lúc nào cũng đông nhân viên chạy ra chạy vô tội nghiệp luôn mọi thứ viên chiên ở đây có đủ mỗi lần thèm an vặt là mình lại ra đây giá khá rẻ 1 dĩa an phủ phê luôn 4 đứa chia ra mỗi đứa 50k lúc trước an thì quán chỉ bán ở nhà chính thôi riết rồi lấn ra đuờng rồi qua bên quán cơm tấm 3 lùn bán luôn phát triển ghê gớm
6,2db8ae0a,"Quán rất đẹp . Nhạc rất hay . Rất art ! ! Bước vô là thích liền luôn : 3 Đồ uống thì buổi_sáng hôm đó mình đi thì_có cà_phê à , nên uống bạc xỉu đỡ : ' ( phục_vụ dễ_thương . Nói_chung quán yên_tĩnh và đẹp lắm , mình thích nhưng nơi yên_tĩnh như_vậy . Sẽ nhất_định quay lại Người_Tình . À , trên bàn dài có cuốn note bự thiệt bự , dành cho mọi người thích ghi những kỉ_niệm đến quán hay của riêng mình vào ... Có nhiều người vẽ đẹp lắm , hi . Khen mãi quán thôi , chả có gì chê luôn . = ) )",1,quán rất đẹp nhạc rất hay rất art bước vô là thích liền luôn 3 đồ uống thì buổi sáng hôm đó mình đi thì có cà phê à nên uống bạc xỉu đỡ phục vụ dễ thương nói chung quán yên tĩnh và đẹp lắm mình thích nhưng nơi yên tĩnh như vậy sẽ nhất định quay lại người tình à trên bàn dài có cuốn note bự thiệt bự dành cho mọi người thích ghi những kỉ niệm đến quán hay của riêng mình vào có nhiều người vẽ đẹp lắm hi khen mãi quán thôi chả có gì chê luôn
7,5a840374,"Gemini thì nhạc bao hay òy : 3 Dj đánh là lên nóc luôn ... Lên là lên là lên đụng nóc luônnn ... Cập_nhậ

In [ ]:
data.to_csv('train_preprocess.csv', columns=['id', 'preprocess_text', 'class'])
!cp "train_preprocess.csv" $output_path

# Test Section

In [ ]:
train_preprocess_path = "/content/drive/MyDrive/ML/BERT/train_preprocess_unsegment.csv"

prep = pd.read_csv(train_preprocess_path)
data['teacher_preprocess_text'] = prep['preprocess_text']

In [ ]:
import numpy as np

data['compare'] = np.where((data['preprocess_text'] == data['teacher_preprocess_text']), 1, 0)
data['compare'].value_counts()

1    15699
0    11301
Name: compare, dtype: int64

In [ ]:
# Test 1 sample

id = 14

ori_text = data['text'][id]
trans_text = preprocess_text(ori_text)
true_text = data['teacher_preprocess_text'][id]

print(f"TRA >>{trans_text}<<")
print(f"TRU >>{true_text}<<")
print()
print(f"ORI >>{ori_text}<<")

TRA >>mình đã đến đây với ý định đến cho biết nhưng không ngờ ber và đồ ăn ở đây lại tuyệt với đến vậy không gian cũng rất là đẹp với sự quan tâm chu đáo của nhân viên mình đã tìm được loại ber mình thích trong hơn 10 loại ber ở đây và đây cũng chính là lần đầu tiên mình được uống những chai ber ngon như thế ở việt nam cuối cùng mình cũng tìm được một địa điểm để thư giản tại sài thành xô bồ này<<
TRU >>mình đã đến đây với ý định đến cho biết nhưng không ngờ ber và đồ ăn ở đây lại tuyệt với đến vậy không gian cũng rất là đẹp với sự quan tâm chu đáo của nhân viên mình đã tìm được loại ber mình thích trong hơn 10 loại ber ở đây và đây cũng chính là lần đầu tiên mình được uống những chai ber ngon như thế ở việt nam cuối cùng mình cũng tìm được một địa điểm để thư giản tại sài thành xô bồ này<<

ORI >>Mi ̀ nh đa ̃ đê ́ n đây vơ ́ i y ́ đi ̣ nh đê ́ n cho biê ́ t nhưng không ngơ ̀ beer va ̀ đô ̀ ăn ơ ̉ đây la ̣ i tuyê ̣ t vơ ́ i đê ́ n vâ ̣ y . Không_gian cu ̃ ng râ ́ t la ̀ đe ̣ p . Vơ ́ i

In [ ]:
# Show random

data.sample(20)

,id,text,class,preprocess_text,teacher_preprocess_text,compare
14863,968ed229,"Mình ăn_ở đây 3 lần.có một lần hết than , nhân_viên vô_ý_thức cầm xẻng than trước mặt đĩa thức_ăn . bụi than rơi vào bát . lúc đó đồ_ăn cũng gần hết nên ko nói làm_gì . đồ_ăn thấy ngon nhưng cản giác quán đông_đúc , ko đc sạch_sẽ . mình khoái nhất ăn đồ nướng . mà ăn xong mùi đồ ăn_bám đầy người",0,mình ăn ở đây 3 lần có một lần hết than nhân viên vô ý thức cầm xẻng than trước mặt đĩa thức ăn bụi than rơi vào bát lúc đó đồ ăn cũng gần hết nên ko nói làm gì đồ ăn thấy ngon nhưng cản giác quán đông đúc ko đc sạch sẽ mình khoái nhất ăn đồ nướng mà ăn xong mùi đồ ăn bám đầy người,mình ăn ở đây 3 lần có một lần hết than nhân viên vô ý thức cầm xẻng than trước mặt đĩa thức ăn bụi than rơi vào bát lúc đó đồ ăn cũng gần hết nên ko nói làm gì đồ ăn thấy ngon nhưng cản giác quán đông đúc ko đc sạch sẽ mình khoái nhất ăn đồ nướng mà ăn xong mùi đồ ăn bám đầy người,0
13547,ee8dcaca,"Tình_hình là đã thử quán này , quá_xá là ưng_ý ! ! ! Món ăn ngon , lạ , giá tốt , phục_vụ dễ_thương , chủ quán cute , không_gian ấm_áp !\nLúc tính tiền anh chủ còn ưu_đãi 5% vì lý_do khách cute đáng yêu ko cưỡng được . haha . Tuy ko có mê trai nhưng lần sau có dịp lên Đà_Lạt sẽ ghé lại ủng_hộ . !\nHy_vọng các bạn giữ vững phong_độ nhé !\nKhoái món cá_sấu nướng , dzú heo nướng , ... mà nói_tóm_lại là hình_như món nào cũng khoái ! Bị bỏ bùa rồi !",1,tình hình là đã thử quán này quá xá là ưng ý món ăn ngon lạ giá tốt phục vụ dễ thương chủ quán cute không gian ấm áp lúc tính tiền anh chủ còn ưu đãi 5 vì lý do khách cute đáng yêu ko cưỡng được haha tuy ko có mê trai nhưng lần sau có dịp lên đà lạt sẽ ghé lại ủng hộ hy vọng các bạn giữ vững phong độ nhé khoái món cá sấu nướng dzú heo nướng mà nói tóm lại là hình như món nào cũng khoái bị bỏ bùa rồi,tình hình là đã thử quán này quá xá là ưng ý món ăn ngon lạ giá tốt phục vụ dễ thương chủ quán cute không gian ấm áp lúc tính tiền anh chủ còn ưu đãi 5 vì lý do khách cute đáng yêu ko cưỡng được haha tuy ko có mê trai nhưng lần sau có dịp lên đà lạt sẽ ghé lại ủng hộ hy vọng các bạn giữ vững phong độ nhé khoái món cá sấu nướng dzú heo nướng mà nói tóm lại là hình như món nào cũng khoái bị bỏ bùa rồi,1
26269,a7ee7bd3,"Trời nắng , đi xe đừng ngoài nhìn không thấy bảo về đâu , đừng hồi lâu thì mới có người mặc đồ bình_thường , ra kêu để xe đó đi ... Thấy không ổn\nKhông_gian thoáng mát , giá_cả củng tốt\nGọi một ly sinh tốt chanh dây , làm ra một ly quá đẹp , nhưng mình không biết ở đây sao chứ mình từng đi rất nhiều quán sinh_tố rồi , chưa thấy quán nào làm sinh_tố chanh dây mà hầu_như không lượt được_một_cái hột nào ra , uống xong nhai như đang nhai cát vậy\nKhông uống nổi ngụm thứ 2\n* góp_ý xíu : tính tiền tại quầy thì củng đúng , nhưng tính tiền thì cho khách xin cái bill và số để nếu có vấn_đề gì dễ ăn_nói hơn ^ ^",0,trời nắng đi xe đừng ngoài nhìn không thấy bảo về đâu đừng hồi lâu thì mới có người mặc đồ bình thường ra kêu để xe đó đi thấy không ổn không gian thoáng mát giá cả củng tốt gọi một ly sinh tốt chanh dây làm ra một ly quá đẹp nhưng mình không biết ở đây sao chứ mình từng đi rất nhiều quán sinh tố rồi chưa thấy quán nào làm sinh tố chanh dây mà hầu như không lượt được một cái hột nào ra uống xong nhai như đang nhai cát vậy không uống nổi ngụm thứ 2 góp ý xíu tính tiền tại quầy thì củng đúng nhưng tính tiền thì cho khách xin cái bil và số để nếu có vấn đề gì dễ ăn nói hơn,trời nắng đi xe đừng ngoài nhìn không thấy bảo về đâu đừng hồi lâu thì mới có người mặc đồ bình thường ra kêu để xe đó đi thấy không ổn không gian thoáng mát giá cả củng tốt gọi một ly sinh tốt chanh dây làm ra một ly quá đẹp nhưng mình không biết ở đây sao chứ mình từng đi rất nhiều quán sinh tố rồi chưa thấy quán nào làm sinh tố chanh dây mà hầu như không lượt được một cái hột nào ra uống xong nhai như đang nhai cát vậy không uống nổi ngụm thứ 2 góp ý xíu tính tiền tại quầy thì củng đúng nhưng tính tiền thì cho khách xin cái bil và số để nếu có vấn đề gì dễ ăn 